In [2]:
import pandas as pd
import json

In [14]:
# Carregar o arquivo JSON
with open("dados/applicants.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Lista para armazenar os dados achatados
applicants_list = []

# Iterar sobre cada aplicante no JSON
for applicant_id, details in data.items():
    # Criar dicionário base com o ID do aplicante
    flat_dict = {"id_candidato": applicant_id}
       
    # Iterar sobre cada categoria de informações
    for category, sub_data in details.items():
        if isinstance(sub_data, dict):
            # Se for um dicionário, achatá-lo adicionando o nome da categoria como prefixo
            for key, value in sub_data.items():
                flat_dict[f"{category}_{key}"] = value
        else:
            # Se não for um dicionário, adicionar diretamente
            flat_dict[category] = sub_data
       
    applicants_list.append(flat_dict)

# Criar DataFrame
df = pd.DataFrame(applicants_list)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42482 entries, 0 to 42481
Data columns (total 58 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   id_candidato                                      42482 non-null  object
 1   infos_basicas_telefone_recado                     42482 non-null  object
 2   infos_basicas_telefone                            42482 non-null  object
 3   infos_basicas_objetivo_profissional               42482 non-null  object
 4   infos_basicas_data_criacao                        42482 non-null  object
 5   infos_basicas_inserido_por                        42482 non-null  object
 6   infos_basicas_email                               42482 non-null  object
 7   infos_basicas_local                               42482 non-null  object
 8   infos_basicas_sabendo_de_nos_por                  42482 non-null  object
 9   infos_basicas_data_atualizac

In [16]:
df.head()

,id_candidato,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,...,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
0,31000,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31001,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31002,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31003,,(11) 98100-1727,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31004,,(11) 92517-2678,,10-11-2021 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,10-11-2021 09:22:03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Verificando porque existem algumas colunas (`cargo_atual_id_ibrati`,`cargo_atual_email_corporativo`,  `cargo_atual_cargo_atual`...) sendo que elas não aparecem por exemplo no primeiro id do json.

Então verifiquei primeiro os que não tem NaN pra poder verificar no json e entender quantos ids tem essa info.

In [9]:
superiores = df[df['cargo_atual_nome_superior_imediato'].notna()]
print(superiores[['id', 'cargo_atual_nome_superior_imediato']].head())

         id cargo_atual_nome_superior_imediato
544   31544                                   
814   31814                                   
946   31946                                   
1501   9342                                   
1634   9475                                   


Bom, somente 5 ids possuem alguma informação nesses campos. Se formos verificar no jason id por id, podemos ver que a maioria da info está nula ou sem nada. Isso indica que talvez possamos remover essas features.

In [12]:
df[df['id'] == '31946']

,id,infos_basicas_telefone_recado,infos_basicas_telefone,infos_basicas_objetivo_profissional,infos_basicas_data_criacao,infos_basicas_inserido_por,infos_basicas_email,infos_basicas_local,infos_basicas_sabendo_de_nos_por,infos_basicas_data_atualizacao,...,cargo_atual_id_ibrati,cargo_atual_email_corporativo,cargo_atual_cargo_atual,cargo_atual_projeto_atual,cargo_atual_cliente,cargo_atual_unidade,cargo_atual_data_admissao,cargo_atual_data_ultima_promocao,cargo_atual_nome_superior_imediato,cargo_atual_email_superior_imediato
946,31946,,(11) 98868-1966,Consultora SAP SD,10-12-2021 09:57:59,Yasmin da Rosa,josé_pedro_nunes@gmail.com,"Guarulhos, São Paulo",Anúncio,03-02-2022 18:19:31,...,,,,,,,31-12-1969,31-12-1969,,


## Tabela prospects

In [19]:
# Carregar o arquivo JSON de prospects
with open("dados/prospects.json", "r", encoding="utf-8") as file:
    data_prospects = json.load(file)

# Lista para armazenar os dados achatados
prospects_list = []

# Iterar sobre cada prospect no JSON
for prospect_id, details in data_prospects.items():
    # Criar dicionário base com o ID do prospect
    flat_dict = {"id": prospect_id}
    
    # Iterar sobre cada categoria de informações
    for category, sub_data in details.items():
        if isinstance(sub_data, dict):
            # Se for um dicionário, achatá-lo adicionando o nome da categoria como prefixo
            for key, value in sub_data.items():
                flat_dict[f"{category}_{key}"] = value
        else:
            # Se não for um dicionário, adicionar diretamente
            flat_dict[category] = sub_data
    
    prospects_list.append(flat_dict)

# Criar DataFrame
df_prospects = pd.DataFrame(prospects_list)
df_prospects


df

,id_prospect,titulo,modalidade,prospects
0,4530,CONSULTOR CONTROL M,,"[{'nome': 'José Vieira', 'codigo': '25632', 's..."
1,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,"[{'nome': 'Sra. Yasmin Fernandes', 'codigo': '..."
2,4532,,,[]
3,4533,2021-2605708-Microfocus Application Life Cycle...,,"[{'nome': 'Arthur Almeida', 'codigo': '26338',..."
4,4534,2021-2605711-Microfocus QTP - UFT Automation T...,,"[{'nome': 'Ana Luiza Vieira', 'codigo': '26361..."
...,...,...,...,...
14217,14218,Java Support Analyst,,"[{'nome': 'Dr. Vitor Hugo Silva', 'codigo': '4..."
14218,14219,,,[]
14219,14220,Consultor Sênior Especialista SAP LES-TRA - 1433,,"[{'nome': 'Ana Cardoso', 'codigo': '16828', 's..."
14220,14221,Consultor Sênior Oracle EPM FCCS - 1434,,"[{'nome': 'Maria Eduarda Cassiano', 'codigo': ..."
